# Extensive data quality checks
# Contain the data basic summary information
created on 2020/05/27, updated on 2020/12/17


In [ ]:
from google.cloud import bigquery
import pandas as pd
import IPython
import re
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
from plotly import figure_factory as ff
from plotly.subplots import make_subplots

# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# Dash
import dash
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output

#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
###If Querying the concept table in the : Can use parametrization as follows: 

cdm_project_id = 'som-rit-phi-starr-prod'
#cdm_project_id = 'som-rit-starr-training'
work_project_id = 'som-rit-phi-starr-dev'
#work_project_id = 'som-nero-phi-rit'
work_dataset_id = 'Jaden_extensive_qc'

In [ ]:
client = bigquery.Client(project=work_project_id)

In [ ]:
client1 = bigquery.Client(project=cdm_project_id)
project1=client1.project
datasets=list(client1.list_datasets())
print(project1)

In [ ]:
lst=[]
##List Datasets and Tables in project
if datasets:
    for dataset in datasets:  # API request(s)
        lst.append(format(dataset.dataset_id))

In [ ]:
prefix='starr_omop_cdm5_deid_1pcent_lite'
num=-5

In [ ]:
cdm_dataset_id_list=[]
for i in lst:
    if i.startswith(prefix) and re.search(r'\d{4}_\d{2}_\d{2}$',i):
        cdm_dataset_id_list.append(i)
cdm_dataset_id_list=cdm_dataset_id_list[num:]
cdm_dataset_id_list

## Demographics

In [ ]:
person_counts=pd.DataFrame(columns=['date','counts'])
gender_counts=pd.DataFrame(columns=['date','gender','counts','percent'])
race_counts=pd.DataFrame(columns=['date','race','counts','percent'])
eth_counts=pd.DataFrame(columns=['date','ethnicity','counts','percent'])
for cdm_dataset_id in cdm_dataset_id_list:
    print(cdm_dataset_id)
    date=cdm_dataset_id.split('_')[-3]+'_'+cdm_dataset_id.split('_')[-2]+'_'+cdm_dataset_id.split('_')[-1]
    # Number of people
    sql="""
    select 
     '{date}' as date,
     row_count as counts
     from 
     `{cdm_project_id}.{cdm_dataset_id}.__TABLES__`
     where table_id='person'
    """.format_map({'cdm_project_id': cdm_project_id,
                    'cdm_dataset_id': cdm_dataset_id,
                    'date': date})
    query_job =client.query(sql)
    countds=query_job.to_dataframe()
    person_counts=person_counts.append(countds)   

    # Number of people by gender
    sql="""
        SELECT
        '{date}' as date,
        concept.CONCEPT_NAME AS gender,
        COUNT(person.person_ID) AS counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.person` as person
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        ON
        person.GENDER_CONCEPT_ID = concept.CONCEPT_ID
        GROUP BY concept.CONCEPT_NAME
    """.format_map({'cdm_project_id': cdm_project_id,
                    'cdm_dataset_id': cdm_dataset_id,
                    'date': date})
    query_job =client.query(sql)
    countds=query_job.to_dataframe()
    countds=countds[countds['gender']!='No matching concept']
    countds['percent']=round(countds['counts']/sum(countds['counts'])*100,1)
    gender_counts=gender_counts.append(countds)

    # Number of people by race
    sql="""
        SELECT
        '{date}' as date,
        concept.CONCEPT_NAME AS race,
        COUNT(person.person_ID) AS counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.person` as person
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        ON
        person.race_CONCEPT_ID = concept.CONCEPT_ID
        GROUP BY concept.CONCEPT_NAME
    """.format_map({'cdm_project_id': cdm_project_id,
                    'cdm_dataset_id': cdm_dataset_id,
                    'date': date})
    query_job =client.query(sql)
    countds=query_job.to_dataframe()
    countds=countds[countds['race']!='No matching concept']
    countds['percent']=round(countds['counts']/sum(countds['counts'])*100,1)
    race_counts=race_counts.append(countds)

    # Number of people by ethnicity
    sql="""
        SELECT
        '{date}' as date,
        concept.CONCEPT_NAME AS ethnicity,
        COUNT(person.person_ID) AS counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.person` as person
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        ON
        person.ethnicity_CONCEPT_ID = concept.CONCEPT_ID
        GROUP BY concept.CONCEPT_NAME
    """.format_map({'cdm_project_id': cdm_project_id,
                    'cdm_dataset_id': cdm_dataset_id,
                    'date': date})
    query_job =client.query(sql)
    countds=query_job.to_dataframe()
    countds=countds[countds['ethnicity']!='No matching concept']
    countds['percent']=round(countds['counts']/sum(countds['counts'])*100,1)
    eth_counts=eth_counts.append(countds)


In [ ]:
totalpatfig = px.bar(person_counts, x="date", y="counts",text='counts')
totalpatfig .update_traces(texttemplate='%{text:.2s}',textposition='outside')
totalpatfig.show()

In [ ]:
genderfig = px.bar(gender_counts, x="date", y="counts", color="gender",text='percent')
genderfig.update_layout(barmode='group')
genderfig.show()

In [ ]:
racefig = px.bar(race_counts, x="date", y="counts", color="race")
racefig.update_layout(barmode='group')
racefig.update_layout(legend=dict(orientation="h", yanchor="bottom",y=1.02,xanchor="right",x=1))
racefig.show()

In [ ]:
ethfig = px.bar(eth_counts, x="date", y="counts", color="ethnicity",text='percent')
ethfig.update_layout(barmode='group')
ethfig.show()

## Mapping rate (defined as percent of *_concept_id is not 0)

In [ ]:
def listtostring(s):
    txt=[]
    for domain in s:
        txt.append('COUNTIF('+domain+'<>0) as maprate_'+domain)
    listToStr = ','.join([str(elem) for elem in txt])
    #print(listToStr) 
    return(listToStr)

In [ ]:
def calculate_maprate(cdm_dataset,cdm_project_id=cdm_project_id):
    sql="""
    select table_name, column_name from
    {cdm_project_id}.{cdm_dataset_id}.INFORMATION_SCHEMA.COLUMNS as a
    join
    {cdm_project_id}.{cdm_dataset_id}.__TABLES__ as b
    on a.table_name=b.table_id
    where row_count!=0 and regexp_contains(column_name,'_concept_id')
    order by table_name, column_name
    """.format_map({'cdm_project_id': cdm_project_id,
                    'cdm_dataset_id': cdm_dataset})
    query_job=client.query(sql)
    temp =query_job.to_dataframe()
    table_list=temp.table_name.unique()
    for table_id in table_list:
        print(table_id)
        domain_list=temp[temp['table_name']==table_id]['column_name']
        mapvars=listtostring(domain_list)
            sql="""
            SELECT
                '{date}' as data_date,
                '{table_name}' AS table_id,
                {mapvars}
                FROM
                `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
            """.format_map({'date':date,
                            'table_id':table_id,
                            'mapvars': mapvars})
            query_job=client.query(sql)
            countds=query_job.to_dataframe()
            #print(countds)
            extraqc =extraqc.append(countds, ignore_index=True)






## Check Person Table

In [ ]:
table_id='person'

In [ ]:
%%time
QUERY= """
       SELECT 
       round(COUNTIF(gender_concept_id <> 0) / COUNT(person_id) * 100,2) AS gender_maprate,
       round(COUNTIF(race_concept_id <> 0) / COUNT(person_id) * 100,2) AS race_maprate,
       round(COUNTIF(ethnicity_concept_id <> 0) / COUNT(person_id) * 100,2) AS ethnicity_maprate,
       round(COUNTIF(gender_source_concept_id <> 0) / COUNT(person_id) * 100,2) AS genders_maprate,
       round(COUNTIF(race_source_concept_id <> 0) / COUNT(person_id) * 100,2) AS races_maprate,
       round(COUNTIF(ethnicity_source_concept_id <> 0) / COUNT(person_id) * 100,2) AS ethnicitys_maprate,
       round(COUNTIF(birth_DATETIME is null)/COUNT(*)*100,2) as birthdt_missrate,
       round(COUNTIF(person_source_value is null)/COUNT(*)*100,2) as person_source_value_null_rate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t1_tab1 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Gender<b>","<b>Race<b>","<b>Ethnicity<b>","<b>Gender source<b>","<b>Race source<b>","<b>Ethnicity source<b>"),
                align='center',
                font_size=15,
                height=20),
    cells=dict(values=[countds.gender_maprate, countds.race_maprate, countds.ethnicity_maprate,countds.genders_maprate, countds.races_maprate, countds.ethnicitys_maprate],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=25))
])

t1_tab1.update_layout(
    width=800,
    height=80,
    margin=dict(t=0,b=0)
)

t1_tab1.show()

In [ ]:
t1_tab2 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Birth date<b>","<b>Person source value<b>"),
                align='center',
                font_size=15,
                height=20),
    cells=dict(values=[countds.birthdt_missrate, countds.person_source_value_null_rate],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=25))
])

t1_tab2.update_layout(
    width=800,
    height=80,
    margin=dict(t=0,b=0)
)

t1_tab2.show()

#### Check summary statistics of birth year, birth month, birth day, age in years

In [ ]:
%%time
QUERY= """
       SELECT
       'year of birth' as variable,
       min(year_of_birth) as minval,
       max(year_of_birth) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       select
       'month of birth' as variable,
       min(month_of_birth) as minval,
       max(month_of_birth) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       select
       'day of birth' as variable,
       min(day_of_birth) as minval,
       max(day_of_birth) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)

In [ ]:
summaryds=query_job.to_dataframe()
#summaryds=summaryds.reset_index(drop=True)
summaryds=summaryds.sort_values(by=['variable'],ascending=True)
#table = ff.create_table(summaryds)
#plotly.offline.iplot(table)

In [ ]:
t1_tab3 = go.Figure(data=[go.Table(
    columnwidth = [1,0.5,0.5],
    
    header=dict(values=("<b>Variable<b>","<b>Min<b>","<b>Max<b>"),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[summaryds.variable,summaryds.minval,summaryds.maxval],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t1_tab3.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t1_tab3.show()

#### Age Distribution

In [ ]:
%%time
QUERY= """
       SELECT
       date_diff(current_date,date(birth_DATETIME),year) as age_yrs
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)

In [ ]:
#ageds['age_yrs'].describe()
ageds=query_job.to_dataframe()
dd=round(ageds['age_yrs'].describe(),1).to_frame()
dd['Statistics']=['Count','Mean',"Standard Deviation","Min","Q1","Median","Q3","Max"]
dd=dd[['Statistics','age_yrs']]
#dd=dd.rename(columns={"age_yrs": "Age in years"})
#table = ff.create_table(dd,height_constant=20)
#plotly.offline.iplot(table)

In [ ]:
t1_tab4 = go.Figure(data=[go.Table(
    columnwidth = [20,5],
    
    header=dict(values=("<b>Descriptive Statistics<b>",""),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[dd.Statistics, dd.age_yrs],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t1_tab4.update_layout(
    width=700,
    height=300,
    margin=dict(t=0,b=0)
)

t1_tab4.show()

In [ ]:
t1_fig1=px.histogram(ageds,x='age_yrs')
t1_fig1.update_layout(
    xaxis_title="Age in years",
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1,
    margin=dict(t=0),
    width=1700,
    height=500
)

t1_fig1.show()

In [ ]:
t1_fig2=px.box(ageds,x='age_yrs')
t1_fig2.update_layout(
    xaxis_title="Age in years",
    title={'text':'Mean:47.4  Standard deviation:25.1',
            'xanchor':'center','yanchor':'top',
          'x':0.5},
    width=1700,
    height=500
)

t1_fig2.show()

#### Gender

In [ ]:
%%time
query="""
        SELECT
        person.GENDER_CONCEPT_ID,
        concept.CONCEPT_NAME AS gender,
        COUNT(person.person_ID) AS counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as person
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        ON
        person.GENDER_CONCEPT_ID = concept.CONCEPT_ID
        GROUP BY
        person.GENDER_CONCEPT_ID,
        concept.CONCEPT_NAME
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
gender = client.query(query).to_dataframe()

In [ ]:
gender=gender[gender['gender']!='No matching concept']
gender['percent']=round(gender['counts']/sum(gender['counts'])*100,1)

In [ ]:
t1_fig3 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t1_fig3.add_trace(go.Table(
    columnwidth = [1,1,1],
    
    header=dict(values=("<b>gender_concept_id<b>","<b>gender<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[gender.GENDER_CONCEPT_ID, gender.gender, gender.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t1_fig3.add_trace(
    go.Bar(
        x=gender['gender'], 
        y=gender['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t1_fig3.update_layout(
    width=1700,
    height=500,
    margin=dict(t=0,b=0)
)

t1_fig3.show()

#### Race

In [ ]:
%%time
query="""
        SELECT
        person.race_concept_id,
        concept.CONCEPT_NAME AS race,
        COUNT(person.person_ID) AS counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as person
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        ON
        person.race_concept_id = concept.CONCEPT_ID                                          
        GROUP BY
        person.race_concept_id,
        concept.CONCEPT_NAME
        order by COUNT(person.person_ID) DESC
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

race = client.query(query).to_dataframe()

In [ ]:
race=race[race['race']!='No matching concept']
race['percent']=round(race['counts']/sum(race['counts'])*100,1)

In [ ]:
t1_fig4 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t1_fig4.add_trace(go.Table(
    columnwidth = [0.5,1,0.5],
    header=dict(values=("<b>race_concept_id<b>","<b>race<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[race.race_concept_id, race.race, race.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t1_fig4.add_trace(
    go.Bar(
        x=race['race'], 
        y=race['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t1_fig4.update_layout(
    width=1700,
    height=500,
    margin=dict(t=0,b=0)
)

t1_fig4.show()

#### Ethnicity

In [ ]:
%%time
query="""
        SELECT
        person.ethnicity_concept_id,
        concept.CONCEPT_NAME AS ethnicity,
        COUNT(person.person_ID) AS counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as person
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        ON
        person.ethnicity_concept_id = concept.CONCEPT_ID
        GROUP BY
        person.ethnicity_concept_id,
        concept.CONCEPT_NAME
        order by COUNT(person.person_ID) DESC
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

eth = client.query(query).to_dataframe()

In [ ]:
eth=eth[eth['ethnicity']!='No matching concept']
eth['percent']=round(eth['counts']/sum(eth['counts'])*100,1)
eth

In [ ]:
t1_fig5 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t1_fig5.add_trace(go.Table(
    columnwidth = [0.8,1,0.5],
    header=dict(values=("<b>ethnicity_concept_id<b>","<b>ethnicity<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[eth.ethnicity_concept_id, eth.ethnicity, eth.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t1_fig5.add_trace(
    go.Bar(
        x=eth['ethnicity'], 
        y=eth['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t1_fig5.update_layout(
    width=1700,
    height=500,
    margin=dict(t=0,b=0)
)

t1_fig5.show()

## Condition_occurrence

In [ ]:
table_id='condition_occurrence'

In [ ]:
%%time
QUERY= """
       SELECT 
       round(COUNTIF(condition_concept_id <> 0) / COUNT(person_id) * 100,2) AS condition_maprate,
       round(COUNTIF(condition_type_concept_id <> 0) / COUNT(person_id) * 100,2) AS conditiontype_maprate,
       round(COUNTIF(condition_source_concept_id <> 0) / COUNT(person_id) * 100,2) AS conditionsource_maprate,
       round(COUNTIF(condition_status_concept_id <> 0) / COUNT(person_id) * 100,2) AS conditionstatus_maprate,
       round(COUNTIF(condition_start_DATETIME is null)/COUNT(*)*100,2) as condition_st_missrate,
       round(COUNTIF(condition_end_DATETIME is null)/COUNT(*)*100,2) as condition_et_missrate,
       round(COUNTIF(stop_reason is null)/COUNT(*)*100,2) as stopreason_missrate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t2_tab1 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Condition<b>","<b>Condition type<b>","<b>Condition source<b>","<b>Condition status<b>"),
                align='center',
                font_size=15,
                height=20),
    cells=dict(values=[countds.condition_maprate,countds.conditiontype_maprate, countds.conditionsource_maprate, countds.conditionstatus_maprate],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=25))
])

t2_tab1.update_layout(
    width=800,
    height=80,
    margin=dict(t=0,b=0)
)

t2_tab1.show()

In [ ]:
t2_tab2 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Condtion start date<b>","<b>Condition end date<b>","<b>Condtion stop reason<b>"),
                align='center',
                font_size=15,
                height=20),
    cells=dict(values=[countds.condition_st_missrate, countds.condition_et_missrate,countds.stopreason_missrate],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=25))
])

t2_tab2.update_layout(
    width=800,
    height=80,
    margin=dict(t=0,b=0)
)

t2_tab2.show()

#### Summary statistics of condition datetime

In [ ]:
%%time
QUERY= """
       SELECT
       'year of condition start time' as variable,
       min(extract(year from condition_start_DATETIME)) as minval,
       max(extract(year from condition_start_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of condition start time' as variable,
       min(extract(month from condition_start_DATETIME)) as minval,
       max(extract(month from condition_start_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of condition start time' as variable,
       min(extract(day from condition_start_DATETIME)) as minval,
       max(extract(day from condition_start_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'year of condition end time' as variable,
       min(extract(year from condition_end_DATETIME)) as minval,
       max(extract(year from condition_end_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of condition end time' as variable,
       min(extract(month from condition_end_DATETIME)) as minval,
       max(extract(month from condition_end_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of condition end time' as variable,
       min(extract(day from condition_end_DATETIME)) as minval,
       max(extract(day from condition_end_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       order by variable
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
summaryds=query_job.to_dataframe()
#summaryds=summaryds.reset_index(drop=True)
summaryds=summaryds.sort_values(by=['variable'],ascending=True)

In [ ]:
t2_tab3 = go.Figure(data=[go.Table(
    columnwidth = [1,0.5,0.5],
    
    header=dict(values=("<b>Variable<b>","<b>Min<b>","<b>Max<b>"),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[summaryds.variable,summaryds.minval,summaryds.maxval],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t2_tab3.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t2_tab3.show()

#### Counts of conditions

In [ ]:
%%time
query="""
        SELECT
        condition.condition_concept_id,
        concept.concept_name as condition, 
        count(condition.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as condition
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on condition.condition_concept_id = concept.concept_ID
        group by condition_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

condition = client.query(query).to_dataframe()

In [ ]:
condition=condition.sort_values(by=['counts'],ascending=False)
condition['percent']=round(condition['counts']/sum(condition['counts'])*100,1)
topconditions=condition.iloc[0:10,]
topconditions

In [ ]:
t2_fig1 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t2_fig1.add_trace(go.Table(
    columnwidth = [0.8,1,0.5],
    header=dict(values=("<b>condition_concept_id<b>","<b>condition<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[topconditions.condition_concept_id, topconditions.condition, topconditions.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t2_fig1.add_trace(
    go.Bar(
        x=topconditions['condition'], 
        y=topconditions['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t2_fig1.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t2_fig1.show()

#### Condition type counts

In [ ]:
%%time
query="""
        SELECT
        condition.condition_type_concept_id,
        concept.concept_name as condition_type, 
        count(condition.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as condition
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on condition.condition_type_concept_id = concept.concept_ID
        group by condition_type_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

conditiontype = client.query(query).to_dataframe()

In [ ]:
conditiontype=conditiontype.sort_values(by=['counts'],ascending=False)
conditiontype['percent']=round(conditiontype['counts']/sum(conditiontype['counts'])*100,1)
conditiontype

In [ ]:
t2_fig2 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t2_fig2.add_trace(go.Table(
    columnwidth = [0.8,1,0.5],
    header=dict(values=("<b>condition_type_concept_id<b>","<b>condition type<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[conditiontype.condition_type_concept_id, conditiontype.condition_type, conditiontype.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t2_fig2.add_trace(
    go.Bar(
        x=conditiontype['condition_type'], 
        y=conditiontype['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t2_fig2.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t2_fig2.show()

## Device exposure

In [ ]:
table_id='device_exposure'

In [ ]:
%%time
QUERY= """
       SELECT 
       round(COUNTIF(device_concept_id <> 0) / COUNT(person_id) * 100,2) AS device_maprate,
       round(COUNTIF(device_type_concept_id <> 0) / COUNT(person_id) * 100,2) AS devicetype_maprate,
       round(COUNTIF(device_source_concept_id <> 0) / COUNT(person_id) * 100,2) AS devicesource_maprate,
       round(COUNTIF(device_exposure_start_DATETIME is null)/COUNT(*)*100,2) as deviceexp_st_missrate,
       round(COUNTIF(device_exposure_end_DATETIME is null)/COUNT(*)*100,2) as deviceexp_et_missrate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t3_tab1 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Device<b>","<b>Device type<b>","<b>Device source<b>"),
                align='center',
                font_size=15),
    cells=dict(values=[countds.device_maprate,countds.devicetype_maprate,countds.devicesource_maprate],
               fill_color='lavender',
               align='center',
               font_size=15))
])

t3_tab1.update_layout(
    width=800,
    height=80,
    margin=dict(t=0,b=0)
)

t3_tab1.show()

In [ ]:
t3_tab2 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Device start date<b>","<b>Condition end date<b>"),
                align='center',
                font_size=15,
                height=20),
    cells=dict(values=[countds.deviceexp_st_missrate, countds.deviceexp_et_missrate],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=25))
])

t3_tab2.update_layout(
    width=800,
    height=80,
    margin=dict(t=0,b=0)
)

t3_tab2.show()

In [ ]:
%%time
QUERY= """
       SELECT
       'year of device start time' as variable,
       min(extract(year from device_exposure_start_DATETIME )) as minval,
       max(extract(year from device_exposure_start_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of device start time' as variable,
       min(extract(month from device_exposure_start_DATETIME )) as minval,
       max(extract(month from device_exposure_start_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of device start time' as variable,
       min(extract(day from device_exposure_start_DATETIME )) as minval,
       max(extract(day from device_exposure_start_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'year of device end time' as variable,
       min(extract(year from device_exposure_end_DATETIME )) as minval,
       max(extract(year from device_exposure_end_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of device end time' as variable,
       min(extract(month from device_exposure_end_DATETIME )) as minval,
       max(extract(month from device_exposure_end_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of device end time' as variable,
       min(extract(day from device_exposure_end_DATETIME )) as minval,
       max(extract(day from device_exposure_end_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       order by variable
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
summaryds=query_job.to_dataframe()
#summaryds=summaryds.reset_index(drop=True)
summaryds=summaryds.sort_values(by=['variable'],ascending=True)

In [ ]:
t3_tab3 = go.Figure(data=[go.Table(
    columnwidth = [1,0.5,0.5],
    
    header=dict(values=("<b>Variable<b>","<b>Min<b>","<b>Max<b>"),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[summaryds.variable,summaryds.minval,summaryds.maxval],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t3_tab3.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t3_tab3.show()

#### Device

In [ ]:
%%time
query="""
        SELECT
        device.device_concept_id,
        concept.concept_name as device, 
        count(device.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as device
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on device.device_concept_id = concept.concept_ID
        group by device_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

device = client.query(query).to_dataframe()
device=device.sort_values(by=['counts'],ascending=False)
device['percent']=round(device['counts']/sum(device['counts'])*100,1)
topdevice=device.iloc[0:10,]
topdevice

#### Device type

In [ ]:
%%time
query="""
        SELECT
        device.device_type_concept_id,
        concept.concept_name as device, 
        count(device.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as device
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on device.device_type_concept_id = concept.concept_ID
        group by device_type_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

devicetype = client.query(query).to_dataframe()
devicetype=devicetype.sort_values(by=['counts'],ascending=False)
devicetype['percent']=round(devicetype['counts']/sum(devicetype['counts'])*100,1)
devicetype

In [ ]:
t3_fig1 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t3_fig1.add_trace(go.Table(
    columnwidth = [0.8,1,0.5],
    header=dict(values=("<b>condition_type_concept_id<b>","<b>condition type<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[conditiontype.condition_type_concept_id, conditiontype.condition_type, conditiontype.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t3_fig1.add_trace(
    go.Bar(
        x=conditiontype['condition_type'], 
        y=conditiontype['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t3_fig1.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t3_fig1.show()

## Drug exposure

In [ ]:
table_id='drug_exposure'

In [ ]:
%%time
QUERY= """
       SELECT
       round(COUNTIF(drug_concept_id <> 0) / COUNT(person_id) * 100,2) AS drug_maprate,
       round(COUNTIF(drug_type_concept_id <> 0) / COUNT(person_id) * 100,2) AS drugtype_maprate,
       round(COUNTIF(route_concept_id <> 0) / COUNT(*) * 100,2) AS route_maprate,
       round(COUNTIF(drug_source_concept_id <> 0) / COUNT(*) * 100,2) AS drugsource_maprate,
       round(COUNTIF(drug_exposure_start_DATETIME is null)/COUNT(*)*100,2) as drugexp_st_missrate,
       round(COUNTIF(drug_exposure_end_DATETIME is null)/COUNT(*)*100,2) as drugexp_et_missrate,
       round(COUNTIF(stop_reason is null)/COUNT(*)*100,2) as stopreason_missrate,
       round(COUNTIF(refills is null)/COUNT(*)*100,2) as refills_missrate,
       round(COUNTIF(quantity is null)/COUNT(*)*100,2) as quantity_missrate,
       round(COUNTIF(route_source_value is null)/COUNT(*)*100,2) as route_source_missrate,
       round(COUNTIF(dose_unit_source_value is null)/COUNT(*)*100,2) as dose_unit_missrate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t4_tab1 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Drug<b>","<b>Drug type<b>","<b>Route<b>","<b>Drug source<b>"),
                align='center',
                font_size=15),
    cells=dict(values=[countds.drug_maprate,countds.drugtype_maprate,countds.route_maprate,countds.drugsource_maprate],
               fill_color='lavender',
               align='center',
               font_size=15))
])

t4_tab1.update_layout(
    width=800,
    height=90,
    margin=dict(t=0,b=0)
)

t4_tab1.show()

In [ ]:
t4_tab2 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Drug start date<b>","<b>Drug end date<b>","<b>Drug stop reason<b>","<b>Refills<b>","<b>quantity<b>","<b>Route source value<b>","<b>Dose unit source<b>"),
                align='center',
                font_size=15,
                height=20),
    cells=dict(values=[countds.drugexp_st_missrate, countds.drugexp_et_missrate,countds.stopreason_missrate,countds.refills_missrate,countds.quantity_missrate,countds.route_source_missrate,countds.dose_unit_missrate],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=25))
])

t4_tab2.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t4_tab2.show()

In [ ]:
%%time
QUERY= """
       SELECT
       'year of drug start time' as variable,
       min(extract(year from drug_exposure_start_DATETIME )) as minval,
       max(extract(year from drug_exposure_start_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of drug start time' as variable,
       min(extract(month from drug_exposure_start_DATETIME )) as minval,
       max(extract(month from drug_exposure_start_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of drug start time' as variable,
       min(extract(day from drug_exposure_start_DATETIME )) as minval,
       max(extract(day from drug_exposure_start_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'year of drug end time' as variable,
       min(extract(year from drug_exposure_end_DATETIME )) as minval,
       max(extract(year from drug_exposure_end_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of drug end time' as variable,
       min(extract(month from drug_exposure_end_DATETIME )) as minval,
       max(extract(month from drug_exposure_end_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of drug end time' as variable,
       min(extract(day from drug_exposure_end_DATETIME )) as minval,
       max(extract(day from drug_exposure_end_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       order by variable
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
summaryds=query_job.to_dataframe()
summaryds=summaryds.sort_values(by=['variable'],ascending=True)

In [ ]:
t4_tab3 = go.Figure(data=[go.Table(
    columnwidth = [1,0.5,0.5],
    
    header=dict(values=("<b>Variable<b>","<b>Min<b>","<b>Max<b>"),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[summaryds.variable,summaryds.minval,summaryds.maxval],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t4_tab3.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t4_tab3.show()

#### Drug type

In [ ]:
%%time
query="""
        SELECT
        drug.drug_type_concept_id,
        concept.concept_name as drug_type, 
        count(drug.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as drug
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on drug.drug_type_concept_id = concept.concept_ID
        group by drug_type_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

drugtype = client.query(query).to_dataframe()
drugtype=drugtype.sort_values(by='counts',ascending=False)
drugtype['percent']=round(drugtype['counts']/sum(drugtype['counts'])*100,1)
drugtype

In [ ]:
t4_fig1 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t4_fig1.add_trace(go.Table(
    columnwidth = [0.8,1,0.5],
    header=dict(values=("<b>drug_type_concept_id<b>","<b>drug type<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[drugtype.drug_type_concept_id, drugtype.drug_type, drugtype.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t4_fig1.add_trace(
    go.Bar(
        x=drugtype['drug_type'], 
        y=drugtype['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t4_fig1.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t4_fig1.show()

#### Drug route

In [ ]:
%%time
query="""
        SELECT
        drug.route_concept_id,
        concept.concept_name as route_type, 
        count(drug.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as drug
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on drug.route_concept_id = concept.concept_ID
        group by route_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

drugroute = client.query(query).to_dataframe()

In [ ]:
drugroute=drugroute[drugroute['route_type']!='No matching concept']
drugroute['percent']=round(drugroute['counts']/sum(drugroute['counts'])*100,1)
drugroute=drugroute.sort_values(by='counts',ascending=False)
topdrugroute=drugroute.iloc[0:10,]

In [ ]:
t4_fig2 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t4_fig2.add_trace(go.Table(
    columnwidth = [0.8,1,0.5],
    header=dict(values=("<b>route_concept_id<b>","<b>route type<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[topdrugroute.route_concept_id, topdrugroute.route_type, topdrugroute.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t4_fig2.add_trace(
    go.Bar(
        x=topdrugroute['route_type'], 
        y=topdrugroute['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t4_fig2.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t4_fig2.show()

#### Drug

In [ ]:
%%time
query="""
        SELECT
        drug.drug_concept_id,
        concept.concept_name as drug, 
        count(drug.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as drug
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on drug.drug_concept_id = concept.concept_ID
        group by drug_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

drug = client.query(query).to_dataframe()
drug=drug[drug['drug']!='No matching concept']
drug=drug.sort_values(by='counts',ascending=False)
drug['percent']=round(drug['counts']/sum(drug['counts'])*100,1)
topdrug=drug.iloc[0:10,]
topdrug

In [ ]:
t4_fig3 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t4_fig3.add_trace(go.Table(
    columnwidth = [0.8,1,0.5],
    header=dict(values=("<b>drug_concept_id<b>","<b>drug<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[topdrug.drug_concept_id, topdrug.drug, topdrug.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t4_fig3.add_trace(
    go.Bar(
        x=topdrug['drug'], 
        y=topdrug['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t4_fig3.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t4_fig3.show()

## Measurement

In [ ]:
table_id='measurement'

In [ ]:
%%time
QUERY= """
       SELECT
       round(COUNTIF(measurement_concept_id <> 0) / COUNT(*) * 100,2) AS measure,
       round(COUNTIF(measurement_type_concept_id <> 0) / COUNT(*) * 100,2) AS measuretype,
       round(COUNTIF(operator_concept_id <> 0) / COUNT(*) * 100,2) AS operator,
       round(COUNTIF(value_as_concept_id <> 0) / COUNT(*) * 100,2) AS valueas,
       round(COUNTIF(unit_concept_id <> 0) / COUNT(*) * 100,2) AS unit,
       round(COUNTIF(measurement_source_concept_id <> 0) / COUNT(*) * 100,2) AS measurement_source,
       round(COUNTIF(measurement_DATETIME is null)/COUNT(*)*100,2) as measuredate_missrate,
       round(COUNTIF(value_as_number is null)/COUNT(*)*100,2) as value_missrate,
       round(COUNTIF(range_low is null)/COUNT(*)*100,2) as low_missrate,
       round(COUNTIF(range_high is null)/COUNT(*)*100,2) as high_missrate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t5_tab1 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Measurement<b>","<b>Measurement type<b>","<b>Operator<b>","<b>Value as<b>","<b>Unit<b>","<b>Source<b>"),
                align='center',
                font_size=15),
    cells=dict(values=[countds.measure,countds.measuretype,countds.operator,countds.valueas,countds.unit,countds.measurement_source],
               fill_color='lavender',
               align='center',
               font_size=15))
])

t5_tab1.update_layout(
    width=1100,
    height=90,
    margin=dict(t=0,b=1)
)

t5_tab1.show()

In [ ]:
t5_tab2 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Measurement date<b>","<b>Value as number<b>","<b>Range low<b>","<b>Range high<b>"),
                align='center',
                font_size=15,
                height=20),
    cells=dict(values=[countds.measuredate_missrate, countds.value_missrate,countds.low_missrate,countds.high_missrate],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=25))
])

t5_tab2.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t5_tab2.show()

In [ ]:
%%time
QUERY= """
       SELECT
       'year of measurement date' as variable,
       min(extract(year from measurement_DATETIME)) as minval,
       max(extract(year from measurement_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of measurement date' as variable,
       min(extract(month from measurement_DATETIME)) as minval,
       max(extract(month from measurement_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of measurement date' as variable,
       min(extract(day from measurement_DATETIME)) as minval,
       max(extract(day from measurement_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
summaryds=query_job.to_dataframe()
#summaryds=summaryds.reset_index(drop=True)
summaryds=summaryds.sort_values(by=['variable'],ascending=True)

In [ ]:
t5_tab3 = go.Figure(data=[go.Table(
    columnwidth = [1,0.5,0.5],
    
    header=dict(values=("<b>Variable<b>","<b>Min<b>","<b>Max<b>"),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[summaryds.variable,summaryds.minval,summaryds.maxval],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t5_tab3.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t5_tab3.show()

#### Measurement type

In [ ]:
%%time
query="""
        SELECT
        m.measurement_type_concept_id,
        concept.concept_name as measurement_type, 
        count(m.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as m
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on m.measurement_type_concept_id = concept.concept_ID
        group by measurement_type_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

measurementtype = client.query(query).to_dataframe()

In [ ]:
#measurement=measurement[measurement['measurement']!='No matching concept']
measurementtype=measurementtype.sort_values(by='counts',ascending=False)
measurementtype['percent']=round(measurementtype['counts']/sum(measurementtype['counts'])*100,1)
measurementtype

In [ ]:
t5_fig1 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t5_fig1.add_trace(go.Table(
    columnwidth = [1,1,0.5],
    header=dict(values=("<b>measurement_type_concept_id<b>","<b>measurement<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[measurementtype.measurement_type_concept_id, measurementtype.measurement_type, measurementtype.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t5_fig1.add_trace(
    go.Bar(
        x=measurementtype['measurement_type'], 
        y=measurementtype['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t5_fig1.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t5_fig1.show()

In [ ]:
%%time
query="""
        SELECT
        m.measurement_concept_id,
        concept.concept_name as measurement, 
        count(m.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as m
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on m.measurement_concept_id = concept.concept_ID
        group by measurement_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

measurement = client.query(query).to_dataframe()

In [ ]:
measurement=measurement[measurement['measurement']!='No matching concept']
measurement=measurement.sort_values(by='counts',ascending=False)
measurement['percent']=round(measurement['counts']/sum(measurement['counts'])*100,1)
topmeasurement=measurement.iloc[0:10,]
topmeasurement

In [ ]:
t5_fig2 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t5_fig2.add_trace(go.Table(
    columnwidth = [1,1,0.5],
    header=dict(values=("<b>measurement_concept_id<b>","<b>measurement<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[topmeasurement.measurement_concept_id, topmeasurement.measurement, topmeasurement.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t5_fig2.add_trace(
    go.Bar(
        x=topmeasurement['measurement'], 
        y=topmeasurement['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t5_fig2.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t5_fig2.show()

## Procedure occurrence

In [ ]:
table_id='procedure_occurrence'

In [ ]:
%%time
QUERY= """
       SELECT 
       round(COUNTIF(procedure_concept_id <> 0) / COUNT(*) * 100,2) AS procedure,
       round(COUNTIF(procedure_type_concept_id <> 0) / COUNT(*) * 100,2) AS proceduretype,
       round(COUNTIF(procedure_source_concept_id <> 0) / COUNT(*) * 100,2) AS proceduresource,
       round(COUNTIF(modifier_concept_id <> 0) / COUNT(*) * 100,2) AS modifier,
       round(COUNTIF(procedure_DATETIME is null)/COUNT(*)*100,2) as proceduredate_missrate,
       round(COUNTIF(quantity is null)/COUNT(*)*100,2) as quantity_missrate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t6_tab1 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Procedure<b>","<b>Procedure type<b>","<b>Procedure source<b>","<b>Modifier<b>"),
                align='center',
                font_size=15),
    cells=dict(values=[countds.procedure,countds.proceduretype,countds.proceduresource,countds.modifier],
               fill_color='lavender',
               align='center',
               font_size=15))
])

t6_tab1.update_layout(
    width=800,
    height=90,
    margin=dict(t=0,b=1)
)

t6_tab1.show()

In [ ]:
t6_tab2 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Procedure date<b>","<b>Quantity<b>"),
                align='center',
                font_size=15,
                height=20),
    cells=dict(values=[countds.proceduredate_missrate, countds.quantity_missrate],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=25))
])

t6_tab2.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t6_tab2.show()

In [ ]:
%%time
QUERY= """
       SELECT
       'year of procedure date' as variable,
       min(extract(year from procedure_DATETIME)) as minval,
       max(extract(year from procedure_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of procedure date' as variable,
       min(extract(month from procedure_DATETIME)) as minval,
       max(extract(month from procedure_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of procedure date' as variable,
       min(extract(day from procedure_DATETIME)) as minval,
       max(extract(day from procedure_DATETIME)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
summaryds=query_job.to_dataframe()
#summaryds=summaryds.reset_index(drop=True)
summaryds=summaryds.sort_values(by=['variable'],ascending=True)

In [ ]:
t6_tab3 = go.Figure(data=[go.Table(
    columnwidth = [1,0.5,0.5],
    
    header=dict(values=("<b>Variable<b>","<b>Min<b>","<b>Max<b>"),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[summaryds.variable,summaryds.minval,summaryds.maxval],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t6_tab3.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t6_tab3.show()

In [ ]:
%%time
query="""
        SELECT
        m.procedure_type_concept_id,
        concept.concept_name as procedure_type, 
        count(m.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as m
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on m.procedure_type_concept_id = concept.concept_ID
        group by procedure_type_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

proceduretype = client.query(query).to_dataframe()

In [ ]:
#measurement=measurement[measurement['measurement']!='No matching concept']
proceduretype=proceduretype.sort_values(by='counts',ascending=False)
proceduretype['percent']=round(proceduretype['counts']/sum(proceduretype['counts'])*100,1)
proceduretype

In [ ]:
t6_fig1 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t6_fig1.add_trace(go.Table(
    columnwidth = [1,1,0.5],
    header=dict(values=("<b>procedure_type_concept_id<b>","<b>procedure type<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[proceduretype.procedure_type_concept_id, proceduretype.procedure_type, proceduretype.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t6_fig1.add_trace(
    go.Bar(
        x=proceduretype['procedure_type'], 
        y=proceduretype['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t6_fig1.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t6_fig1.show()

In [ ]:
%%time
query="""
        SELECT
        m.procedure_concept_id,
        concept.concept_name as procedure, 
        count(m.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as m
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on m.procedure_concept_id = concept.concept_ID
        group by procedure_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

procedure = client.query(query).to_dataframe()

In [ ]:
procedure=procedure[procedure['procedure']!='No matching concept']
procedure=procedure.sort_values(by='counts',ascending=False)
procedure['percent']=round(procedure['counts']/sum(procedure['counts'])*100,1)
topprocedure=procedure.iloc[0:10,]

In [ ]:
t6_fig2 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t6_fig2.add_trace(go.Table(
    columnwidth = [1,1,0.5],
    header=dict(values=("<b>procedure_concept_id<b>","<b>procedure<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[topprocedure.procedure_concept_id, topprocedure.procedure, topprocedure.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t6_fig2.add_trace(
    go.Bar(
        x=topprocedure['procedure'], 
        y=topprocedure['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t6_fig2.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t6_fig2.show()

## Visit detail

In [ ]:
table_id='visit_detail'

In [ ]:
%%time
QUERY= """
       SELECT 
       round(COUNTIF(visit_detail_concept_id <> 0) / COUNT(*) * 100,2) AS visit,
       round(COUNTIF(visit_detail_type_concept_id <> 0) / COUNT(*) * 100,2) AS visittype,
       round(COUNTIF(visit_detail_source_concept_id <> 0) / COUNT(*) * 100,2) AS visitsource,
       round(COUNTIF(admitting_source_concept_id <> 0) / COUNT(*) * 100,2) AS admitted,
       round(COUNTIF(discharge_to_concept_id <> 0) / COUNT(*) * 100,2) AS discharge,
       round(COUNTIF(visit_detail_start_datetime is null)/COUNT(*)*100,2) as visitsdt_missrate,
       round(COUNTIF(visit_detail_end_datetime is null)/COUNT(*)*100,2) as visitedt_missrate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t7_tab1 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Visit detail<b>","<b>Visit detail type<b>","<b>Visit detail source<b>","<b>Admitting source<b>","<b>Discharge<b>"),
                align='center',
                font_size=15),
    cells=dict(values=[countds.visit,countds.visittype,countds.visitsource,countds.admitted,countds.discharge],
               fill_color='lavender',
               align='center',
               font_size=15))
])

t7_tab1.update_layout(
    width=800,
    height=90,
    margin=dict(t=0,b=1)
)

t7_tab1.show()

In [ ]:
t7_tab2 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Visit start date<b>","<b>Visit end date<b>"),
                align='center',
                font_size=15,
                height=20),
    cells=dict(values=[countds.visitsdt_missrate, countds.visitedt_missrate],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=25))
])

t7_tab2.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t7_tab2.show()

In [ ]:
%%time
QUERY= """
       SELECT
       'year of visit start time' as variable,
       min(extract(year from visit_detail_start_datetime )) as minval,
       max(extract(year from visit_detail_start_datetime )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of visit start time' as variable,
       min(extract(month from visit_detail_start_datetime )) as minval,
       max(extract(month from visit_detail_start_datetime )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of visit start time' as variable,
       min(extract(day from visit_detail_start_datetime )) as minval,
       max(extract(day from visit_detail_start_datetime )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'year of visit end time' as variable,
       min(extract(year from visit_detail_end_datetime )) as minval,
       max(extract(year from visit_detail_end_datetime )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of visit end time' as variable,
       min(extract(month from visit_detail_end_datetime )) as minval,
       max(extract(month from visit_detail_end_datetime )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of visit end time' as variable,
       min(extract(day from visit_detail_end_datetime )) as minval,
       max(extract(day from visit_detail_end_datetime )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       order by variable
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
summaryds=query_job.to_dataframe()
summaryds=summaryds.sort_values(by=['variable'],ascending=True)

In [ ]:
t7_tab3 = go.Figure(data=[go.Table(
    columnwidth = [1,0.5,0.5],
    
    header=dict(values=("<b>Variable<b>","<b>Min<b>","<b>Max<b>"),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[summaryds.variable,summaryds.minval,summaryds.maxval],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t7_tab3.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t7_tab3.show()

In [ ]:
%%time
query="""
        SELECT
        m.visit_detail_type_concept_id,
        concept.concept_name as visit_type, 
        count(m.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as m
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on m.visit_detail_type_concept_id = concept.concept_ID
        group by visit_detail_type_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

visittype = client.query(query).to_dataframe()

In [ ]:
#measurement=measurement[measurement['measurement']!='No matching concept']
visittype=visittype.sort_values(by='counts',ascending=False)
visittype['percent']=round(visittype['counts']/sum(visittype['counts'])*100,1)
visittype

In [ ]:
t7_tab4 = go.Figure(data=[go.Table(
    columnwidth = [1,0.5,0.5],
    
    header=dict(values=("<b>Visit_detail_type_concept_id<b>","<b>visit_type<b>","<b>counts<b>","<b>percent<b>"),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[visittype.visit_detail_type_concept_id,visittype.visit_type,visittype.counts,visittype.percent],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t7_tab4.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t7_tab4.show()

In [ ]:
%%time
query="""
        SELECT
        m.visit_detail_concept_id,
        concept.concept_name as visit, 
        count(m.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as m
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on m.visit_detail_concept_id = concept.concept_ID
        group by visit_detail_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

visit = client.query(query).to_dataframe()

In [ ]:
visit=visit[visit['visit']!='No matching concept']
visit=visit.sort_values(by='counts',ascending=False)
visit['percent']=round(visit['counts']/sum(visit['counts'])*100,1)
topvisit=visit.iloc[0:10,]
topvisit

In [ ]:
t7_fig1 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t7_fig1.add_trace(go.Table(
    columnwidth = [1,1,0.5],
    header=dict(values=("<b>visit_detail_concept_id<b>","<b>visit<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[topvisit.visit_detail_concept_id, topvisit.visit, topvisit.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t7_fig1.add_trace(
    go.Bar(
        x=topvisit['visit'], 
        y=topvisit['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t7_fig1.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t7_fig1.show()

## Care site

In [ ]:
table_id='care_site'

In [ ]:
%%time
QUERY= """
       SELECT 
       round(COUNTIF(place_of_service_concept_id <> 0) / COUNT(*) * 100,2) AS place_of_service,
       round(COUNTIF(care_site_name is null)/COUNT(*)*100,2) as caresitename_missrate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t8_tab1 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Place of service<b>","<b>Care site name<b>"),
                align='center',
                font_size=15),
    cells=dict(values=[countds.place_of_service,countds.caresitename_missrate],
               fill_color='lavender',
               align='center',
               font_size=15))
])

t8_tab1.update_layout(
    width=800,
    height=90,
    margin=dict(t=0,b=1)
)

t8_tab1.show()

## Concept ancestor

In [ ]:
table_id='concept_ancestor'

In [ ]:
%%time
QUERY= """
       SELECT 
       round(COUNTIF(ancestor_concept_id <> 0) / COUNT(*) * 100,2) AS ancestor,
       round(COUNTIF(descendant_concept_id <> 0) / COUNT(*) * 100,2) AS descendant,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t8_tab2 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Ancestor<b>","<b>Descendant<b>"),
                align='center',
                font_size=15),
    cells=dict(values=[countds.ancestor,countds.descendant],
               fill_color='lavender',
               align='center',
               font_size=15))
])

t8_tab2.update_layout(
    width=800,
    height=90,
    margin=dict(t=0,b=1)
)

t8_tab2.show()

## Death

In [ ]:
table_id='death'

In [ ]:
%%time
QUERY= """
       SELECT
       count(*) as number_rows,
       count(distinct(person_id)) as number_people_died,
       round(COUNTIF(death_type_concept_id <> 0) / COUNT(*) * 100,2) AS death_type,
       round(COUNTIF(cause_concept_id <> 0) / COUNT(*) * 100,2) AS death_cause,
       round(COUNTIF(cause_source_concept_id <> 0) / COUNT(*) * 100,2) AS death_cause_source,
       round(COUNTIF(death_date is not null)/COUNT(*)*100,2) as deathdate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t8_tab3 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Death type<b>","<b>Death cause<b>","<b>Death cause source<b>","<b>Death date<b>"),
                align='center',
                font_size=15),
    cells=dict(values=[countds.death_type,countds.death_cause,countds.death_cause_source,countds.deathdate],
               fill_color='lavender',
               align='center',
               font_size=15))
])

t8_tab3.update_layout(
    width=800,
    height=90,
    margin=dict(t=0,b=1)
)

t8_tab3.show()

In [ ]:
%%time
QUERY= """
       SELECT
       'year of death date' as variable,
       min(extract(year from death_DATE)) as minval,
       max(extract(year from death_DATE)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of death date' as variable,
       min(extract(month from death_DATE)) as minval,
       max(extract(month from death_DATE)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of death date' as variable,
       min(extract(day from death_DATE)) as minval,
       max(extract(day from death_DATE)) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
summaryds=query_job.to_dataframe()
#summaryds=summaryds.reset_index(drop=True)
summaryds=summaryds.sort_values(by=['variable'],ascending=True)

In [ ]:
t8_tab4 = go.Figure(data=[go.Table(
    columnwidth = [1,0.5,0.5],
    
    header=dict(values=("<b>Variable<b>","<b>Min<b>","<b>Max<b>"),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[summaryds.variable,summaryds.minval,summaryds.maxval],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t8_tab4.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t8_tab4.show()

## Domain

In [ ]:
table_id='domain'

In [ ]:
%%time
QUERY= """
       SELECT
       round(COUNTIF(domain_concept_id <> 0) / COUNT(*) * 100,2) AS domain,
       round(COUNTIF(domain_name is not null)/COUNT(*)*100,2) as domainname_missrate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t8_tab5 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Domain<b>","<b>Domain name<b>"),
                align='center',
                font_size=15),
    cells=dict(values=[countds.domain,countds.domainname_missrate],
               fill_color='lavender',
               align='center',
               font_size=15))
])

t8_tab5.update_layout(
    width=800,
    height=90,
    margin=dict(t=0,b=1)
)

t8_tab5.show()

## Drug strength

In [ ]:
table_id='drug_strength'

In [ ]:
%%time
QUERY= """
       SELECT
       round(COUNTIF(ingredient_concept_id <> 0) / COUNT(*) * 100,2) AS ingredient,
       round(COUNTIF(numerator_unit_concept_id <> 0) / COUNT(*) * 100,2) AS numerator_unit,
       round(COUNTIF(denominator_unit_concept_id <> 0) / COUNT(*) * 100,2) AS denominator_unit,
       round(COUNTIF(valid_start_DATE is null)/COUNT(*)*100,2) as validstartdate_missrate,
       round(COUNTIF(valid_end_DATE is null)/COUNT(*)*100,2) as validenddate_missrate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t8_tab6 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Drug ingredient<b>","<b>Numerator unit<b>","<b>Denominator unit<b>"),
                align='center',
                font_size=15),
    cells=dict(values=[countds.ingredient,countds.numerator_unit, countds.denominator_unit],
               fill_color='lavender',
               align='center',
               font_size=15))
])

t8_tab6.update_layout(
    width=800,
    height=90,
    margin=dict(t=0,b=1)
)

t8_tab6.show()

In [ ]:
t8_tab7 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Valid start date<b>","<b>Valid end date<b>"),
                align='center',
                font_size=15,
                height=20),
    cells=dict(values=[countds.validstartdate_missrate, countds.validenddate_missrate],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=25))
])

t8_tab7.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t8_tab7.show()

In [ ]:
%%time
QUERY= """
       SELECT
       'year of valid start date' as variable,
       min(extract(year from valid_start_DATE )) as minval,
       max(extract(year from valid_start_DATE )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of valid start date' as variable,
       min(extract(month from valid_start_DATE )) as minval,
       max(extract(month from valid_start_DATE )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of valid start date' as variable,
       min(extract(day from valid_start_DATE )) as minval,
       max(extract(day from valid_start_DATE )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'year of valid end date' as variable,
       min(extract(year from valid_end_DATE )) as minval,
       max(extract(year from valid_end_DATE )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of valid end date' as variable,
       min(extract(month from valid_end_DATE )) as minval,
       max(extract(month from valid_end_DATE )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of valid end date' as variable,
       min(extract(day from valid_end_DATE )) as minval,
       max(extract(day from valid_end_DATE )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       order by variable
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
summaryds=query_job.to_dataframe()
summaryds=summaryds.sort_values(by=['variable'],ascending=True)

In [ ]:
t8_tab8 = go.Figure(data=[go.Table(
    columnwidth = [1,0.5,0.5],
    
    header=dict(values=("<b>Variable<b>","<b>Min<b>","<b>Max<b>"),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[summaryds.variable,summaryds.minval,summaryds.maxval],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t8_tab8.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t8_tab8.show()

## Obsersevation

In [ ]:
table_id='observation'

In [ ]:
%%time
QUERY= """
       SELECT
       round(COUNTIF(observation_concept_id <> 0) / COUNT(*) * 100,2) AS observation,
       round(COUNTIF(observation_type_concept_id <> 0) / COUNT(*) * 100,2) AS observation_type,
       round(COUNTIF(value_as_concept_id <> 0) / COUNT(*) * 100,2) AS observation_value,
       round(COUNTIF(qualifier_concept_id <> 0) / COUNT(*) * 100,2) AS observation_qualifier,
       round(COUNTIF(observation_source_concept_id <> 0) / COUNT(*) * 100,2) AS observation_source_value,
       round(COUNTIF(observation_DATETIME is not null)/COUNT(*)*100,2) as observationdate_rate
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
countds=query_job.to_dataframe()

In [ ]:
t8_tab9 = go.Figure(data=[go.Table(
    header=dict(values=("<b>Observation<b>","<b>Observation type<b>","<b>Observation value<b>","<b>Observation qualifier<b>","<b>Observation source value<b>","<b>Observation date<b>"),
                align='center',
                font_size=15),
    cells=dict(values=[countds.observation,countds.observation_type,countds.observation_value,countds.observation_qualifier,countds.observation_source_value,countds.observationdate_rate],
               fill_color='lavender',
               align='center',
               font_size=15))
])

t8_tab9.update_layout(
    width=1000,
    height=90,
    margin=dict(t=0,b=1)
)

t8_tab9.show()

In [ ]:
%%time
QUERY= """
       SELECT
       'year of observation date' as variable,
       min(extract(year from observation_DATETIME )) as minval,
       max(extract(year from observation_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'month of observation date' as variable,
       min(extract(month from observation_DATETIME )) as minval,
       max(extract(month from observation_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       UNION ALL
       SELECT
       'day of observation date' as variable,
       min(extract(day from observation_DATETIME )) as minval,
       max(extract(day from observation_DATETIME )) as maxval,
       FROM `{cdm_project_id}.{cdm_dataset_id}.{table_id}`
       order by variable
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})
#print(QUERY)
query_job =client.query(QUERY)
summaryds=query_job.to_dataframe()
summaryds=summaryds.sort_values(by=['variable'],ascending=True)

In [ ]:
t8_tab10 = go.Figure(data=[go.Table(
    columnwidth = [1,0.5,0.5],
    
    header=dict(values=("<b>Variable<b>","<b>Min<b>","<b>Max<b>"),
                align=('left','center'),
                font_size=15,
                height=25),

    cells=dict(values=[summaryds.variable,summaryds.minval,summaryds.maxval],
               fill_color='lavender',
               align=('left','center'),
               font_size=15,
               height=25))
])

t8_tab10.update_layout(
    width=800,
    height=120,
    margin=dict(t=0,b=0)
)

t8_tab10.show()

In [ ]:
%%time
query="""
        SELECT
        m.observation_concept_id,
        concept.concept_name as observation, 
        count(m.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as m
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on m.observation_concept_id = concept.concept_ID
        group by observation_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

obs = client.query(query).to_dataframe()

In [ ]:
#measurement=measurement[measurement['measurement']!='No matching concept']
obs=obs.sort_values(by='counts',ascending=False)
obs['percent']=round(obs['counts']/sum(obs['counts'])*100,1)
topobs=obs.iloc[0:10,]
topobs

In [ ]:
t8_fig1 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t8_fig1.add_trace(go.Table(
    columnwidth = [1,1,0.5],
    header=dict(values=("<b>observation_concept_id<b>","<b>observation<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[topobs.observation_concept_id, topobs.observation, topobs.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t8_fig1.add_trace(
    go.Bar(
        x=topobs['observation'], 
        y=topobs['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t8_fig1.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t8_fig1.show()

In [ ]:
%%time
query="""
        SELECT
        m.observation_type_concept_id,
        concept.concept_name as observation_type, 
        count(m.person_id) as counts
        FROM
        `{cdm_project_id}.{cdm_dataset_id}.{table_id}` as m
        INNER JOIN
        `{cdm_project_id}.{cdm_dataset_id}.concept` as concept
        on m.observation_type_concept_id = concept.concept_ID
        group by observation_type_concept_id, concept.concept_name
       """.format_map({'cdm_project_id': cdm_project_id,
                        'cdm_dataset_id': cdm_dataset_id,
                        'table_id':table_id})

obstype = client.query(query).to_dataframe()

In [ ]:
#measurement=measurement[measurement['measurement']!='No matching concept']
obstype=obstype.sort_values(by='counts',ascending=False)
obstype['percent']=round(obstype['counts']/sum(obstype['counts'])*100,1)
obstype

In [ ]:
t8_fig2 = make_subplots(
    rows=1, cols=2,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},{"type": "bar"}]]
)

t8_fig2.add_trace(go.Table(
    columnwidth = [1,1,0.5],
    header=dict(values=("<b>observation_type_concept_id<b>","<b>observation type<b>","<b>counts<b>"),
                align='center',
                font_size=15,
                height=30),
    cells=dict(values=[obstype.observation_type_concept_id, obstype.observation_type, obstype.counts],
               fill_color='lavender',
               align='center',
               font_size=15,
               height=30)),
    row=1,col=1                       
)

t8_fig2.add_trace(
    go.Bar(
        x=obstype['observation_type'], 
        y=obstype['percent'], 
        xaxis='x2', 
        yaxis='y2',
        marker=dict(color='mediumslateblue')
    ),
    row=1,col=2
)

t8_fig2.update_layout(
    width=1700,
    height=600,
    margin=dict(t=0,b=0)
)

t8_fig2.show()

## Make Web application using Dash

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash('YourAppExample',external_stylesheets=external_stylesheets)

app.layout = html.Div([
    
    html.H1(children='Extensive data quality checking-2020/07/04 Starr Omop Deid 1 percent',style={'textAlign':'center'}),
    
    dcc.Tabs(id="tabs", value='tab-1', children=[
        dcc.Tab(label='Person', value='tab-1'),
        dcc.Tab(label='Condition occurrence', value='tab-2'),
        dcc.Tab(label='Device exposure',value='tab-3'),
        dcc.Tab(label='Drug exposure',value='tab-4'),
        dcc.Tab(label='Measurement',value='tab-5'),
        dcc.Tab(label='Procedure occurrence',value='tab-6'),
        dcc.Tab(label='Visit detail',value='tab-7'),
        dcc.Tab(label='Other tables',value='tab-8')
    ]),
    html.Div(id='tabs-content')
])

@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])

def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            
            html.H3('Concept id mapping rates'),
            
            dcc.Graph(id='Maprate',
                     figure=t1_tab1
            ),
            
            html.H3('Missing rates'),
            
            dcc.Graph(id='missrate',
                     figure=t1_tab2
            ),
            
            html.H3('Range of birth date'),
            
            dcc.Graph(id='missrate',
                     figure=t1_tab3
            ),
            
            html.H3('Age distribution'),
            
            dcc.Graph(
                id='AgedHistogram',
                figure=t1_fig1
            ),

            dcc.Graph(
                id='Agedboxplot',
                figure=t1_fig2
            ),
            
            html.H3('Gender'),
            
            dcc.Graph(id='gender',
                figure=t1_fig3
            ),
            
            html.H3('Race'),
            
            dcc.Graph(id='race',
                figure=t1_fig4
            ),
            
            html.H3('Ethnicity'),
            
            dcc.Graph(id='ethnicity',
                figure=t1_fig5
            )
            
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Concept id mapping rates'),
            
            dcc.Graph(id='Maprate',
                figure=t2_tab1
            ),
            
            html.H3('Missing rates'),
            
            dcc.Graph(id='missrate',
                figure=t2_tab2
            ),
            
            html.H3('Range of condition date'),
            
            dcc.Graph(id='daterange',
                figure=t2_tab3
            ),
            
            html.H3('Top 10 conditions'),
            
            dcc.Graph(id='top10cond',
                figure=t2_fig1
            ),
            
            html.H3('Condition type'),
            
            dcc.Graph(id='condtype',
                figure=t2_fig2
            ),
            
        ])
    elif tab == 'tab-3':
        return html.Div([
            html.H3('Concept id mapping rates'),
            
            dcc.Graph(id='Maprate',
                figure=t3_tab1
            ),
            
            html.H3('Missing rates'),
            
            dcc.Graph(id='missrate',
                figure=t3_tab2
            ),
            
            html.H3('Range of device date'),
            
            dcc.Graph(id='daterange',
                figure=t3_tab3
            )
        ])
    elif tab == 'tab-4':
        return html.Div([
            html.H3('Concept id mapping rates'),
            
            dcc.Graph(id='Maprate',
                figure=t4_tab1
            ),
            
            html.H3('Missing rates'),
            
            dcc.Graph(id='missrate',
                figure=t4_tab2
            ),
            
            html.H3('Range of device date'),
            
            dcc.Graph(id='daterange',
                figure=t4_tab3
            ),
            
            html.H3('Drug type'),
            
            dcc.Graph(id='drugtype',
                figure=t4_fig1
            ),
            
            html.H3('Drug route'),
            
            dcc.Graph(id='drugroute',
                figure=t4_fig2
            ),
            
            html.H3('Top 10 drugs'),
            
            dcc.Graph(id='druglist',
                figure=t4_fig3
            )
        ])
    elif tab == 'tab-5':
        return html.Div([
            html.H3('Concept id mapping rates'),
            
            dcc.Graph(id='Maprate',
                figure=t5_tab1
            ),
            
            html.H3('Missing rates'),
            
            dcc.Graph(id='missrate',
                figure=t5_tab2
            ),
            
            html.H3('Range of measurement date'),
            
            dcc.Graph(id='daterange',
                figure=t5_tab3
            ),
            
            html.H3('Measurement type'),
            
            dcc.Graph(id='type',
                figure=t5_fig1
            ),
            
            html.H3('Top 10 measurement'),
            
            dcc.Graph(id='list',
                figure=t5_fig2
            )
        ]) 
    elif tab == 'tab-6':
        return html.Div([
            html.H3('Concept id mapping rates'),
            
            dcc.Graph(id='Maprate',
                figure=t6_tab1
            ),
            
            html.H3('Missing rates'),
            
            dcc.Graph(id='missrate',
                figure=t6_tab2
            ),
            
            html.H3('Range of procedure date'),
            
            dcc.Graph(id='daterange',
                figure=t6_tab3
            ),
            
            html.H3('Procedure type'),
            
            dcc.Graph(id='type',
                figure=t6_fig1
            ),
            
            html.H3('Top 10 procedures'),
            
            dcc.Graph(id='list',
                figure=t6_fig2
            )
        ])
    elif tab == 'tab-7':
        return html.Div([
            html.H3('Concept id mapping rates'),
            
            dcc.Graph(id='Maprate',
                figure=t7_tab1
            ),
            
            html.H3('Missing rates'),
            
            dcc.Graph(id='missrate',
                figure=t7_tab2
            ),
            
            html.H3('Range of procedure date'),
            
            dcc.Graph(id='daterange',
                figure=t7_tab3
            ),
            
            html.H3('Procedure type'),
            
            dcc.Graph(id='type',
                figure=t7_tab4
            ),
            
            html.H3('Top 10 visit'),
            
            dcc.Graph(id='list',
                figure=t7_fig1
            )
        ]) 
    elif tab == 'tab-8':
        return html.Div([
            html.H3('Care site'),
            
            dcc.Graph(id='Maprate',
                figure=t8_tab1
            ),
            
            html.H3('Concept ancestor'),
            
            dcc.Graph(id='Maprate',
                figure=t8_tab2
            ),
            
            html.H3('Death'),
            
            dcc.Graph(id='maprate',
                figure=t8_tab3
            ),
            
            dcc.Graph(id='datet',
                figure=t8_tab4
            ),
            
            html.H3('Domain'),
            
            dcc.Graph(id='type',
                figure=t8_tab5
            ),
            
            html.H3('Drug strength'),
            
            dcc.Graph(id='list',
                figure=t8_tab6
            ),
            
            dcc.Graph(id='list',
                figure=t8_tab7
            ),
            
            dcc.Graph(id='list',
                figure=t8_tab8
            ),
            
            html.H3('Observation'),
            
            dcc.Graph(id='list',
                figure=t8_tab9
            ),
            
            dcc.Graph(id='list',
                figure=t8_tab10
            ),
            
            dcc.Graph(id='list',
                figure=t8_fig1
            ),
            
            dcc.Graph(id='list',
                figure=t8_fig2
            )
        ])
    
app.run_server(mode='external')